In [21]:
from crocoddyl import *
import pinocchio as pin
import numpy as np
from crocoddyl.diagnostic import displayTrajectory

In [22]:
# LOAD ROBOT
robot = loadKinton()
robot.initViewer(loadModel=True)
rmodel = robot.model

q0 = rmodel.referenceConfigurations["centered"]
robot.display(q0)
robot.framesForwardKinematics(q0)

In [23]:
# DEFINE TARGET POSITION
target_pos  = np.array([0,0,1])
[ -0.7071068, 0, 0.7071068, 0 ]
target_quat = pin.Quaternion(-1, 0, 1, 0)
target_quat.normalize()

# Plot goal frame
robot.viewer.gui.addXYZaxis('world/framegoal', [1., 0., 0., 1.], .015, 4)
robot.viewer.gui.applyConfiguration('world/framegoal', target_pos.tolist() + [target_quat[0], target_quat[1], target_quat[2], target_quat[3]])
robot.viewer.gui.refresh()

In [24]:
# ACTUATION MODEL
distanceRotorCOG = 0.1525
cf = 6.6e-5
cm = 1e-6
actModel = ActuationModelUAM(robot.model, distanceRotorCOG, cf, cm)

# COST MODEL
# Create a cost model per the running and terminal action model.
runningCostModel = CostModelSum(robot.model, actModel.nu)
terminalCostModel = CostModelSum(robot.model, actModel.nu)

frameName = 'link6'
state = StatePinocchio(robot.model)
SE3ref = pin.SE3()
SE3ref.translation = target_pos.reshape(3,1)
SE3ref.rotation = target_quat.matrix()


wBasePos  = [1]
wBaseOri  = [5]
wArmPos   = [1]
wBaseVel  = [10]
wBaseRate = [10]
wArmVel   = [10] 
stateWeights   = np.array(wBasePos * 3 + wBaseOri * 3 + wArmPos * (robot.model.nv - 6) + wBaseVel * robot.model.nv)
controlWeights = np.array([0.1]*4 + [100]*6)

goalTrackingCost = CostModelFramePlacement(rmodel,
                                           frame=rmodel.getFrameId(frameName),
                                           ref=SE3ref,
                                           nu =actModel.nu)

xRegCost = CostModelState(rmodel, 
                          state, 
                          ref=state.zero(), 
                          nu=actModel.nu,
                          activation=ActivationModelWeightedQuad(stateWeights))
uRegCost = CostModelControl(rmodel, 
                            nu=robot.
                            model.nv-2,
                            activation = ActivationModelWeightedQuad(controlWeights))
uLimCost = CostModelControl(rmodel, 
                            nu=robot.
                            model.nv-2,
                            activation = ActivationModelInequality(np.array([0.1, 0.1, 0.1, 0.1, -1, -1, -1, -1, -1, -1]), 
                                                                np.array([5, 5, 5, 5, 1, 1, 1, 1, 1, 1])))

# Then let's add the running and terminal cost functions
runningCostModel.addCost(name="pos", weight=0.1, cost=goalTrackingCost)
runningCostModel.addCost(name="regx", weight=1e-4, cost=xRegCost)
runningCostModel.addCost(name="regu", weight=1e-6, cost=uRegCost)
# runningCostModel.addCost(name="limu", weight=1e-3, cost=uLimCost)
terminalCostModel.addCost(name="pos", weight=0, cost=goalTrackingCost)

# DIFFERENTIAL ACTION MODEL
runningModel = IntegratedActionModelEuler(DifferentialActionModelUAM(robot.model, actModel, runningCostModel))
terminalModel = IntegratedActionModelEuler(DifferentialActionModelUAM(robot.model, actModel, terminalCostModel))

In [25]:
# DEFINING THE SHOOTING PROBLEM & SOLVING

# Defining the time duration for running action models and the terminal one
dt = 5e-2
runningModel.timeStep = dt

# For this optimal control problem, we define 250 knots (or running action
# models) plus a terminal knot
T = 20
q0 = rmodel.referenceConfigurations["centered"].copy()
v0 = pin.utils.zero(rmodel.nv)
x0 = m2a(np.concatenate([q0, v0]))
rmodel.defaultState = x0.copy()

problem = ShootingProblem(x0, [runningModel] * T, terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
fddp = SolverFDDP(problem)
fddp.callback = [CallbackDDPVerbose()]
fddp.callback.append(CallbackDDPLogger())

us0 = [
    m.differential.quasiStatic(d.differential, rmodel.defaultState)
    if isinstance(m, IntegratedActionModelEuler) else np.zeros(0)
    for m, d in zip(fddp.problem.runningModels, fddp.problem.runningDatas)]
xs0 = [problem.initialState]*len(fddp.models())

# Solving it with the DDP algorithm
#fddp.solve(init_xs=xs0, init_us=us0)
fddp.solve()

iter 	 cost 	      stop 	    grad 	  xreg 	      ureg 	 step 	 feas
   0  9.08198e+00  4.56473e-01  1.97554e+01  1.00000e-09  1.00000e-09   0.5000     0
   1  4.04502e+00  2.73160e-01  1.52749e+01  1.00000e-09  1.00000e-09   0.5000     0
   2  2.95386e+00  9.48071e-02  4.97010e+00  1.00000e-09  1.00000e-09   0.2500     0
   3  2.46366e+00  5.13997e-02  2.99065e+00  1.00000e-09  1.00000e-09   0.5000     0
   4  1.82967e+00  4.92640e-02  1.93473e+00  1.00000e-09  1.00000e-09   0.5000     0
   5  1.76305e+00  1.16694e-02  6.93346e-01  1.00000e-09  1.00000e-09   0.5000     0
   6  1.56127e+00  1.93517e-02  5.77282e-01  1.00000e-09  1.00000e-09   0.5000     0
   7  1.51972e+00  4.57215e-03  2.26322e-01  1.00000e-09  1.00000e-09   0.5000     0
   8  1.47284e+00  6.55408e-03  1.88064e-01  1.00000e-09  1.00000e-09   0.5000     0
   9  1.42210e+00  3.88011e-03  1.41680e-01  1.00000e-09  1.00000e-09   0.5000     0
iter 	 cost 	      stop 	    grad 	  xreg 	      ureg 	 step 	 feas
  10  1.39035e

([array([0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0.2, 1.7, 2. , 0. , 0. ,
         0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]),
  array([-3.57544605e-03, -7.44668560e-03,  4.49365233e-01,  1.79681024e-02,
         -4.45620815e-03, -3.01835956e-01,  9.53180122e-01,  6.46208456e-01,
         -1.27364727e-02,  1.70866626e+00,  1.99610746e+00, -4.22029271e-02,
         -1.35228129e-01,  2.31269481e-04,  2.09342835e-03,  8.98934560e+00,
          7.30155326e-01, -1.81083347e-01, -1.22654650e+01,  1.29241691e+01,
         -4.25472945e+00,  1.73325174e-01, -7.78508082e-02, -8.44058542e-01,
         -2.70456257e+00]),
  array([-2.75335078e-03, -9.70018618e-03,  7.05702573e-01,  3.82496094e-02,
         -2.73270534e-02, -4.96030274e-01,  8.67031814e-01,  1.10225357e+00,
         -1.21343614e-01,  1.66622939e+00,  1.95528582e+00, -9.22865263e-02,
         -1.87437132e-01,  8.35589340e-02,  3.11522254e-01,  5.11765173e+00,
          1.08505275e+00, -7.88122271e-01, -8.49670771e+0

In [26]:
displayTrajectory(robot, fddp.xs, runningModel.timeStep)

In [27]:
pltUAM = PlotUAM(fddp.xs, fddp.us, T, dt, distanceRotorCOG, cf, cm)

fig, axs = pltUAM.plotMotorForces()


ValueError: could not broadcast input array from shape (20) into shape (10)

In [ ]:
import matplotlib.pyplot as plt

fig,axs = pltUAM.plotActuation()


In [ ]:
import matplotlib.pyplot as plt
t = np.arange(0., 1, dt)

fig, axs = plt.subplots(2,2, figsize=(15,10))
fig.suptitle('Motor forces')
axs[0, 0].plot(t,f1)
axs[0, 0].set_title('Motor 1')
axs[0, 1].plot(t,f2)
axs[0, 1].set_title('Motor 2')
axs[1, 0].plot(t,f3)
axs[1, 0].set_title('Motor 3')
axs[1, 1].plot(t,f4)
axs[1, 1].set_title('Motor 4')

plt.figure()
t = np.append(t, 1)
plt.plot(t,Xx,t,Xy,t,Xz)
plt.legend(['x','y','z'])
plt.title('State - Position')
plt.ylabel('Position, [m]')
plt.xlabel('[s]')

plt.figure()
plt.plot(t,Vx,t,Vy,t,Vz)
plt.legend(['x','y','z'])
plt.title('State - Velocity')
plt.ylabel('Velocity, [m/s]')
plt.xlabel('[s]')

plt.show()

In [ ]:
t = np.arange(0,T*dt,dt)
t_state = np.append(t, t[-1]+dt)
t_state